## Q1

In [ ]:
######### Solution #########

class strategy1(base_strategy):
    def __init__(self):
        super().__init__()
        self.df = pd.DataFrame()
        self.buy = False
        self.count = 4
        
    def fit(self, x):
        x = pd.DataFrame(x, index=[0]).set_index('date')
        self.df = pd.concat([self.df, x], axis=0)
    
    def predict(self, x):
        if self.count==4 and self.buy==False:
            self.count = 1
            self.buy = True
            return DIRECTION.BUY
        elif self.count==4 and self.buy==True:
            self.count = 1
            self.buy = False
            return DIRECTION.SELL
        else:
            self.count+=1
            return DIRECTION.HOLD

import numpy as np
class strategy2(base_strategy):
    def __init__(self):
        super().__init__()
        self.prices = deque(maxlen = 10)
        self.buy = False
        self.df = pd.DataFrame()
        self.first = True
        
    def fit(self, x):
        x = pd.DataFrame(x, index=[0]).set_index('date')
        self.df = pd.concat([self.df, x], axis=0)
    
    def predict(self, x):
        self.prices.append(x['price'])
        mavg = np.mean(self.prices)
        if self.first:
            mavg = 500
            self.first=False
        if(mavg>560 and self.buy):
            self.buy = False
            return DIRECTION.SELL
        elif(mavg<559 and not self.buy):
            self.buy = True
            return DIRECTION.BUY
        else:
            return DIRECTION.HOLD

class strategy3(base_strategy):
    def __init__(self):
        super().__init__()
        self.buy = False
        self.df = pd.DataFrame()
        self.model = None
        self.prev_price = 0

    def fit(self, x):
        temp = [1 if x['price'] > self.prev_price else -1,
                x['price'],
                x['open']-x['close'],
                x['high'] - x['low']
            ]
        self.df = pd.concat([self.df, pd.DataFrame([temp])], axis=0, ignore_index=True)
        self.prev_price = x['price']
        if self.df.shape[0]>500:
            X = self.df.iloc[:, 1:]
            Y = self.df.iloc[:, 0]
            self.model = LogisticRegression(random_state=0).fit(X=X.dropna(),y=Y.dropna())
            
    def predict(self, price_update):
        if self.df.shape[0]>500:
            
            X = [price_update['price'],
                    price_update['open']-price_update['close'],
                    price_update['high'] - price_update['low']
                ]
            X = pd.DataFrame([X])
            result = self.model.predict(X)[0]
            if result==1 and self.buy==False:
                self.buy=True
                return DIRECTION.BUY
            elif result==-1 and self.buy:
                self.buy=False
                return DIRECTION.SELL
            else:
                return DIRECTION.HOLD

In [ ]:
######### my version (strategy3 can't pass) #########

#!/bin/python3

import math
import os
import random
import re
import sys
import pandas as pd
from abc import ABC
from collections import deque
from sklearn.linear_model import LogisticRegression

class DIRECTION:
    BUY=1
    SELL=-1
    HOLD = 0

class base_strategy(ABC):
    def predict(self):
        pass
    def fit(self,price):
        pass



#All the class strategy you will need to define will inherit from the class base_strategy
# base_strategy inherits from the class ABC, it doesn't matter what it means. It just prevents
# from creating an instance of the class base_strategy without inheriting from this class (abstract class)
#You will need to use the inheritance for each class declaration
#Please DO NOT forget to call the constructor of the base class
#You will need to define at least 3 methods
# The constructor __init__
# The predict function
# The fit function
# The function predict will need to return either
    # DIRECTION.BUY
    # DIRECTION.SELL
    # DIRECTION.HOLD

# In all the strategies, do not forget that you will always need to start by buying

class strategy1(base_strategy):
    
    def __init__(self):
        super().__init__()
        self.price_list = []
        
    def fit(self,price):
        
        self.price_list.append(price)
        # print(price)
        # print(len(self.price_list))
        
    def predict(self,price):
        
        if len(self.price_list)%4==1 and ((len(self.price_list)-1)/4)%2==0:
            return DIRECTION.BUY
        elif len(self.price_list)%4==1 and ((len(self.price_list)-1)/4)%2==1:
            return DIRECTION.SELL
        else:
            return DIRECTION.HOLD


# reference: homework
class strategy2(base_strategy):
    
    def __init__(self):
        super().__init__()
        self.price_list = []
        self.buy = True
        
    def fit(self,price):
        
        # print(price)
        self.price_list.append(price['price'])
        
    def predict(self,price):
        
        mavg = -1
        
        if len(self.price_list)>=10:
            mavg = sum(self.price_list[-10:])/10
        else:
            mavg = sum(self.price_list)/len(self.price_list)
        
        # print(mavg)
            
        if mavg!=-1:
            if (mavg>560 and not self.buy):
                self.buy = True
                return DIRECTION.SELL
            elif (mavg<559 and self.buy):
                self.buy = False
                return DIRECTION.BUY
            else:
                return DIRECTION.HOLD

    
        
    
# reference: sklearn doc
# I don't understand where is wrong, the output seems so correct lol

class strategy3(base_strategy):
    # hint for the predict function
    # if len(your_dataframe) <= 500, you will hold until you receive enough data
    # store the previous price
    # create your input X
    # create your input Y
    # train your logistic regression this way: LogisticRegression(random_state=0).fit(X=X,y=Y)
    # build your X_test with the latest value then use the function predict
    # if prediction == 1 and you were not in the position you can buy
    # if prediction == -1 and not you were in a long position, you can sell
    # if not you can hold
    def __init__(self):
        super().__init__()
        self.prev_price = 0
        self.price_list = []
        self.buy = True
        
    def fit(self,price):
        label = 0
        
        if price['price'] > self.prev_price:
            label = 1
        else:
            label = -1
        
        price['label'] = label
        self.prev_price = price['price']
        self.price_list.append(price)
        
    def predict(self,price):
        
        if len(self.price_list)<= 500:
            return DIRECTION.HOLD
        
        # print(len(self.price_list))
    
        X = []
        Y = []
        
        for i in range(len(self.price_list)):
            if i!= len(self.price_list)-1:
                p = self.price_list[i]
                x0 = [p['price'],p['open']-p['close'],p['high']-p['low']]
                X.append(x0)
                Y.append(p['label'])
            
        logreg = LogisticRegression(random_state=0).fit(X=X,y=Y)
        
        # label = 0
        
        # if price['price'] > self.prev_price:
        #     label = 1
        # else:
        #     label = -1
        
        # price['label'] = label
    
        y_pred = logreg.predict([[price['price'],price['open']-price['close'],price['high']-price['low']]])
        # print(y_pred)
        
        if y_pred[0]==1 and self.buy:
            self.buy = False
            return DIRECTION.BUY
        elif y_pred[0]==-1 and not self.buy:
            self.buy = True
            return DIRECTION.SELL
        else:
            return DIRECTION.HOLD
    
        
        
        
    


class ForLoopBackTester:
    def __init__(self,strat=None):
        self.list_position=[]
        self.list_cash=[]
        self.list_holdings = []
        self.list_total=[]

        self.long_signal=False
        self.position=0
        self.cash=100000
        self.total=0
        self.holdings=0

        self.market_data_count=0
        self.prev_price = None
        self.statistical_model = None
        self.historical_data = pd.DataFrame(columns=['Trade','Price','OpenClose','HighLow'])
        self.strategy = strat



    def onMarketDataReceived(self,price_update):
        if self.strategy:
            self.strategy.fit(price_update)
            predicted_value = self.strategy.predict(price_update)
        else:
            predicted_value = DIRECTION.HOLD

        if predicted_value==DIRECTION.BUY:
            return 'buy'
        if predicted_value==DIRECTION.SELL:
            return 'sell'
        return 'hold'

    def buy_sell_or_hold_something(self,price_update,action):
        if action == 'buy':
            cash_needed = 10 * price_update['price']
            if self.cash - cash_needed >=0:
                print(str(price_update['date']) +
                      " send buy order for 10 shares price=%.2f" % (price_update['price']))
                self.position += 10
                self.cash -= cash_needed
            else:
                print('buy impossible because not enough cash')


        if action == 'sell':
            position_allowed=10
            if self.position-position_allowed>=-position_allowed:
                print(str(price_update['date'])+
                      " send sell order for 10 shares price=%.2f" % (price_update['price']))
                self.position -= position_allowed
                self.cash -= -position_allowed * price_update['price']
            else:
                print('sell impossible because not enough position')

        self.holdings = self.position * price_update['price']
        self.total = (self.holdings + self.cash)
        # print('%s total=%d, holding=%d, cash=%d' %
        #       (str(price_update['date']),self.total, self.holdings, self.cash))

        self.list_position.append(self.position)
        self.list_cash.append(self.cash)
        self.list_holdings.append(self.holdings)
        self.list_total.append(self.holdings+self.cash)


naive_backtester = None
nb_of_rows = 0

def test1():
    global naive_backtester
    global nb_of_rows
    nb_of_rows=10
    naive_backtester = ForLoopBackTester(strategy1())

def test2():
    global naive_backtester
    global nb_of_rows
    nb_of_rows=50
    naive_backtester = ForLoopBackTester(strategy1())

def test3():
    global naive_backtester
    global nb_of_rows
    nb_of_rows=10
    naive_backtester = ForLoopBackTester(strategy2())

def test4():
    global naive_backtester
    global nb_of_rows
    nb_of_rows=150
    naive_backtester = ForLoopBackTester(strategy2())

def test5():
    global naive_backtester
    global nb_of_rows
    nb_of_rows=600
    naive_backtester = ForLoopBackTester(strategy3())



if __name__ == '__main__':


    func_name = sys.stdin.readline().strip()
    test_func = globals()[func_name]
    test_func()
    market_data_header = input().strip()
    for _ in range(nb_of_rows):
        row = input().strip().split(',')

        date=row[0]
        high=row[1]
        low = row[2]
        closep=row[4]
        openp = row[3]
        volume = row[5]
        price=row[6]

        price_information={'date' : date,
                          'price' : float(price),
                           'high' : float(high),
                           'low': float(low),
                           'close' : float(closep),
                           'open' : float(openp),
                           'volume' : float(volume)}
        action = naive_backtester.onMarketDataReceived(price_information)
        naive_backtester.buy_sell_or_hold_something(price_information,action)



    print("PNL:%.2f" % (naive_backtester.list_total[-1] - 10000))  
        
        


## Q2

In [ ]:
######### Solution #########

import pandas as pd

def processLogs(logs, maxSpan):
    
    # Write your code here
    
    all = pd.DataFrame([l.split() for l in logs], columns = ['userid', 'time', 'action'])
    all['time'] = all['time'].astype(int)
    all['userid'] = all['userid'].astype(int)
    si = all[all['action']=='sign-in']
    so = all[all['action']=='sign-out']
    df = si.merge(so, on='userid', suffixes=['_in', '_out'])
    df['delta'] = df['time_out'] - df['time_in']
    df = df[df['delta']<=maxSpan]
    df = df.sort_values('userid', ignore_index=True)
    df['userid'] = df['userid'].astype(str)
    return df['userid']

In [ ]:
######### my version (time exceeding) #########


#!/bin/python3

import math
import os
import random
import re
import sys


#
# Complete the 'processLogs' function below.
#
# The function is expected to return a STRING_ARRAY.
# The function accepts following parameters:
#  1. STRING_ARRAY logs
#  2. INTEGER maxSpan
#

def processLogs(logs, maxSpan):
    # Write your code here
    
    # user_dict = {}
    
    # for l in logs:
    #     l_list = l.split(" ")
    #     id0 = l_list[0]
    #     time0 = l_list[1]
    #     action0 = l_list[2]
        
    #     if id0 not in list(user_dict.keys()):
    #         user_dict[id0] = [int(time0)]
    #     else:
    #         user_dict[id0].append(int(time0))
            
    
    # result = []
    
    # for key, value in user_dict.items():
        
    #     if len(value)==2:
    #         span = max(value)-min(value)
    #         if span<=maxSpan:
    #             result.append(int(key))
                
    # result = sorted(result)
    
    # for i in range(len(result)):
    #     result[i] = str(result[i])
        
    # # print(result)
    # # print(type(result[0]))
    # return result

    user_dict = {}
    result = []
        
    for l in logs:
        l_list = l.split(" ")
        id0 = l_list[0]
        time0 = int(l_list[1])
        action0 = l_list[2]
        
        
        if id0 not in list(user_dict.keys()):
            user_dict[id0] = time0
        else:
            # user_dict[id0].append(int(time0))
            
            span = max([time0,user_dict[id0]])-min([time0,user_dict[id0]])
            # span = max(user_dict[id0])-min(user_dict[id0])
            if span<=maxSpan:
                result.append(int(id0))
        

    # result = []

    # for key, value in user_dict.items():
        
    #     if len(value)==2:
    #         span = max(value)-min(value)
    #         if span<=maxSpan:
    #             result.append(int(key))
                
    result = sorted(result)

    for i in range(len(result)):
        result[i] = str(result[i])
        
    # print(result)
    # print(type(result[0]))
    return result
            
        

if __name__ == '__main__':
    fptr = open(os.environ['OUTPUT_PATH'], 'w')

    logs_count = int(input().strip())

    logs = []

    for _ in range(logs_count):
        logs_item = input()
        logs.append(logs_item)

    maxSpan = int(input().strip())

    result = processLogs(logs, maxSpan)

    fptr.write('\n'.join(result))
    fptr.write('\n')

    fptr.close()
